In [7]:
# import libraries
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_squared_error,mean_absolute_error,mean_absolute_percentage_error
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import matplotlib.dates as mdates

In [19]:
def  Expected_Revenue(actual_prices,predicted_prices):
    print(len(predicted_prices) - 1)
    predicted_changes = [predicted_prices[i + 1] - predicted_prices[i] for i in range(len(predicted_prices) - 1)]
    expected_revenue = 0
    for i in range(6):  # Assuming a 7-day test period
        if predicted_changes[i] > 0:  # Buy signal
            expected_revenue += (actual_prices[i + 1] - actual_prices[i])
        elif predicted_changes[i] < 0:  # Sell signal
            expected_revenue += (actual_prices[i] - actual_prices[i + 1])
    return expected_revenue

In [20]:
# load dataset
file_list=['BTC-all.csv','BTC-N2Y.csv','BTC-N4Y.csv']

# file_list=['ETH-all.csv','ETH-N2Y.csv','ETH-N4Y.csv']

spilt_list=[0.3,0.2,0.1,7]

files_list=list()
spilts_list=list()
Classifiers_list=list()

train_mae=list()
train_mape=list()
train_rmse=list()
test_mae=list()
test_mape=list()
test_rmse=list()

Expected_revenues=list()

In [21]:
for file in file_list:
    for splits in spilt_list:
        path ='data/'+file
        files_list.append(file)
        spilts_list.append(splits)
        Classifiers_list.append('Prophet')

        df = pd.read_csv(path)
        df=df[['Adj Close','Date']]
        df.rename(columns={'Adj Close': 'y','Date':'ds'}, inplace=True)

        test_spilt=int(df.shape[0]*0.3)
        train_spilt=df.shape[0]-test_spilt

        train_df,test_df=df[:train_spilt],df[train_spilt:]
        print(train_df.shape,test_df.shape)

        m = Prophet()
        m.fit(train_df)

        future = m.make_future_dataframe(periods=(test_df.shape[0]))


        forecast = m.predict(future)

        train_forecast=forecast["yhat"][:train_spilt]
        test_forecast=forecast["yhat"][train_spilt:]
        
        Expected_revenues.append(Expected_Revenue(test_df['y'],test_forecast))
        
        train_mae.append(mean_absolute_error(train_df['y'], train_forecast))
        train_mape.append(mean_squared_error(train_df['y'], train_forecast)**0.5)
        train_rmse.append(mean_absolute_percentage_error(train_df['y'], train_forecast))
        test_mae.append(mean_absolute_error(test_df['y'], test_forecast))
        test_mape.append(mean_squared_error(test_df['y'], test_forecast)**0.5)
        test_rmse.append(mean_absolute_percentage_error(test_df['y'], test_forecast))
     


(2275, 2) (975, 2)


15:51:36 - cmdstanpy - INFO - Chain [1] start processing
15:51:37 - cmdstanpy - INFO - Chain [1] done processing


974


KeyError: 1

In [ ]:
Summary={'Data':files_list,'Classifier':Classifiers_list,'Test Split':spilts_list,
            'train_mae':train_mae,
            'train_mape':train_mape,
            'train_rmse':train_rmse,          
            'test_mae':test_mae,
            'test_mape':test_mape,
            'test_rmse':test_rmse,
            'Expected_Revenue':Expected_revenues
         }

df_Summary = pd.DataFrame(Summary)
df_Summary.to_excel("BTC-Prophet Summary.xlsx")  

In [6]:
# fig2 = m.plot_components(forecast)
